### Загрузим библиотеки

In [ ]:
#!pip install selenium

import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

### Объявим константы

In [ ]:
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
driver_path = "/home/user/Selenium/chromedriver" # путь к chromedriver
base_dir= "/home/user/Selenium/parse/" # укажем директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/116.0" # мой user-agent, взятый на сайте: https://юзерагент.рф, смотреть через браузер Chrome
start_time = time() # время начала выполнения программы

### Зададим несколько вспомогательных функций.
#### Функция get_load_time()

In [ ]:
def get_load_time(article_url, user_agent):
    """Функция для подсчета времени для загрузки каждой новостной веб-страницы (статьи).
        В дальнейшем он может использоваться, например, при планировании времени обхода страниц.

    Args:
        article_url (URL): адрес в интернете, где располагается спарсиваемая статья
        user_agent (text): строка в запросе с информацией о версии ОС, браузера и устройстве.
    Returns:
        int: время загрузки страницы
    """
    # будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
    try:
        # меняем значение заголовка. По умолчанию указано, что это python-код
        headers = {
            "User-Agent": user_agent
        }
        # делаем запрос по url статьи article_url
        response = requests.get(
            article_url, headers=headers, stream=True, timeout=3.000
        )
        # получаем время загрузки страницы
        load_time = response.elapsed.total_seconds()
    except Exception as e:
        print(e)
        load_time = ">3"
    return load_time

### Функция write_to_file

In [ ]:
def write_to_file(output_list, filename, base_dir):
    """Функция для сохранения результатов парсинга в файл.

    Args:
        output_list (list): Список с информацией о спарсиваемых статьях
        filename (text): Имя файла, в который будем сохранять результат
        base_dir (path): Директория, в которую будем сохранять файл
    """
    for row in output_list:
        with open(Path(base_dir).joinpath(filename), "a") as csvfile:
            fieldnames = ["id", "load_time", "rank", "points", "title", "url", "num_comments"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)

### Функция authorization

In [ ]:
def authorization(browser):
    """Функция для прохождения авторизации на сайте, с которого происходит парсинг

    Args:
        browser (WebDriver): Элемент класса WebDriver

    Returns:
        bool: Переменная показывает, успешно ли прошла авторизация
        """
    base_url = "https://news.ycombinator.com"
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            #дождемся появления кнопки "login", и нажмем её
            # затем функция вернет True иначе False
            button = WebDriverWait(browser, 20).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/center/table/tbody/tr[1]/td/table/tbody/tr/td[3]/span/a')))
            button.click()

            # убедимся, необходимые поля готово для ввода и введем логин и пароль
            username = WebDriverWait(browser, 20).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/form[1]/table/tbody/tr[1]/td[2]/input')))
            username.send_keys('piioner')

            password = WebDriverWait(browser, 20).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/form[1]/table/tbody/tr[2]/td[2]/input')))
            password.send_keys('5hOeTtDB')

            # осуществим ввод логина и пароля
            login_button = WebDriverWait(browser, 20).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/form[1]/input[2]')))
            login_button.click()
            # в случае успеха функция вернёт True
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    # в случае неуспеха функция вернёт False
    return False

### Функция connect_to_base

In [ ]:
def connect_to_base(browser, page_number):
    """Прежде чем брать данные с открытой в Selenium страницы, необходимо некоторое время для загрузки на страницу контента.
       Эта функция ожидает загрузку таблицы на страницу некоторое время и возвращает TRUE в случае её доступности.
       WebDriverWait говорит Selenium ждать, пока не выполнится некоторое условие (пока таблица не подгрузится).

    Args:
        browser (WebDriver): Элемент класса WebDriver
        page_number (int): Номер динамической страницы

    Returns:
        _Bool: Информация о том, что загрузка страницы произведена(True) или нет(False).
    """
    base_url = "https://news.ycombinator.com/news?p={}".format(page_number)
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            # ожидаем пока элемент table с id = 'hnmain' будет загружен на страницу
            # затем функция вернет True иначе False
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.ID, "hnmain"))
            )
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    return False

### Функция parse_html()

In [ ]:
def parse_html(html, user_agent):
    """В этой функции мы будем парсить страницу с Beautiful Soup,
        извлекая необходимые атрибуты и записывая их в список.

    Args:
        html (URL): Спарсиваемая страница
        user_agent (text): Строка в запросе с информацией о версии ОС, браузера и устройстве.

    Returns:
        list: Список с информацией о спарсиваемых статьях
    """
    soup = BeautifulSoup(html, "html.parser")
    output_list = []

    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    article = 0
    for tr in tr_blocks:
        article_id = tr.get("id") # id
        article_url = tr.find_all("a")[1]["href"]

        # иногда статья располагается не на внешнем сайте, а на ycombinator
        # тогда article_url у нее не полный, а добавочный, с параметрами.
        # например item?id=200933. Для этих случаев будем добавлять url до полного
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
        load_time = get_load_time(article_url, user_agent)
        # иногда рейтинга может не быть, поэтому воспользуемся try

        try:
            score = soup.find(id=f"score_{article_id}").string
        except Exception as e:
            print(e)
            score = "0 points"

        try:
            attr_value = f"item?id={article_id}"
            comments = soup.find_all('a', {'href': attr_value})[1].string.split()
            num_comments=" ".join(comments)
        except Exception as e:
            print(e)
            comments = "0 comments"

        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": tr.span.string,
            "points": score,
            "title": tr.select("span>a:nth-child(1)")[0].text,
            "url": article_url,
            "num_comments": num_comments
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
    return output_list

## Многопоточность

Запустим многопоточность, немного изменив наш основной код — вынесем его в отдельную функцию и будем запускать её в потоке:

In [ ]:
from concurrent.futures import ThreadPoolExecutor, wait

browser = webdriver.Chrome(
        service=ChromeService(executable_path=driver_path)
    )

#Пройдем авторизацию
if authorization(browser):
    print('Авторизация пройдена')
else:
    print('Авторизация не пройдена')

# Обернём процедуру парсинга страницы в функцию
def run_process(browser, page_number, filename):
    """Функция автоматизирующая парсинг страницы

    Args:
        browser (WebDrive):Элемент класса WebDriver
        page_number (int): Номер динамической страницы
        filename (text): Имя файла, в который будем сохранять результат
    """
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)

        browser.quit()
    else:
        print("Error connecting to hacker news")
        browser.quit()

# Глобальные переменные
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
driver_path = "/home/user/Selenium/chromedriver" # путь к chromedriver
base_dir= "/home/user/Selenium/parse/" # укажем директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/116.0" # мой user-agent, взятый на сайте: https://юзерагент.рф, смотреть через браузер Chrome

# Засечём время выполнения кода
start_time = time()

futures = []

# Запустим процесс парсинга на нескольких потоках одновременно
with ThreadPoolExecutor() as executor:
    for number in range(10):
        futures.append(
            executor.submit(run_process, browser, number, filename)
        )

wait(futures)
end_time = time()
elapsed_time = end_time - start_time
print("Elapsed run time: {} seconds".format(elapsed_time))

Авторизация пройдена
HTTPSConnectionPool(host='medium.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='medium.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='medium.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='medium.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='medium.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='medium.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='medium.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='medium.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.cs.rpi.edu', port=443): Max retries exceeded with url: /academics/courses/spring11/proglang/handouts/lambda-calculus-chapter.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer cert

Чтобы подражать человеку-пользователю, вызывается метод sleep(5),<br>
который вносит задержку на 5 секунд после того, как драйвер подключился к Hacker News.<br>
После загрузки страницы и выполнения sleep(5) драйвер захватывает HTML-код страницы, <br>
который затем передаётся в функцию parse_html().<br>

В результате выполнения кода будет создан файл articles_info.csv, <br>
который будет содержать всю информацию, которую мы спарсили. <br>
Мы можем прочитать этот файл и представить таблицу в виде DataFrame:<br>

In [ ]:
import pandas as pd

articles_data = pd.read_csv(
    '/home/user/Selenium/parse/articles_info.csv',
    names=["id", "load_time", "rank", "points", "title", "url", "num_comments"],
    encoding='utf-8'
)

articles_data.head()

,id,load_time,rank,points,title,url,num_comments
0,37167922,6.525357,151.0,72 points,How the Army tried and failed to build a bicyc...,https://www.armytimes.com/news/your-army/2020/...,63 comments
1,37157811,0.73784,152.0,127 points,"Acronym’s new computer with Asus is bonkers, b...",https://techcrunch.com/2023/08/16/review-acron...,165 comments
2,37179094,0.443781,153.0,223 points,Police are getting DNA data from people who th...,https://theintercept.com/2023/08/18/gedmatch-d...,179 comments
3,37193250,1.337712,154.0,8 points,BSD Now – Episode 520 “4 months BSD”,https://www.bsdnow.tv/520,discuss
4,37167922,6.553725,151.0,72 points,How the Army tried and failed to build a bicyc...,https://www.armytimes.com/news/your-army/2020/...,63 comments
